In [74]:
import os
os.listdir('result')

['test-1-2-sync-compute',
 'test-1-1-sync-sleep',
 'test-1-1-gevent-compute',
 'test-1-4-sync-request',
 'test-4-2-sync-sleep',
 'test-1-4-gaiohttp-sleep',
 'test-4-2-gevent-request',
 'test-2-4-gevent-sleep',
 'test-4-2-gevent-sleep',
 'test-4-2-gaiohttp-compute',
 'test-4-4-gevent-compute',
 'test-4-1-gaiohttp-sleep',
 'test-1-2-sync-request',
 'test-2-2-sync-sleep',
 'test-1-2-gaiohttp-sleep',
 'test-1-1-gevent-request',
 'test-1-4-sync-compute',
 'test-1-4-sync-sleep',
 'test-4-2-gevent-compute',
 'test-4-4-gevent-request',
 'test-4-2-gaiohttp-request',
 'test-4-4-gevent-sleep',
 'test-2-2-gevent-sleep',
 'test-1-2-gevent-compute',
 'test-4-2-sync-request',
 'test-2-4-sync-request',
 'test-1-4-gevent-request',
 'test-2-2-sync-compute',
 'test-4-4-sync-compute',
 'test-2-2-gaiohttp-request',
 'test-4-2-gaiohttp-sleep',
 'test-1-2-gaiohttp-compute',
 'test-1-4-gevent-sleep',
 'test-4-1-gevent-request',
 'test-4-2-sync-compute',
 'test-1-2-gevent-request',
 'test-2-4-sync-compute',
 '

In [2]:
CON = 16
NUM = 32

In [75]:
import re
data = {
    'name': [],
    'num_worker': [],
    'num_threads': [],
    'worker_class': [],
    'endpoint': [],
    'total_time': [],
    'rps': [],
    'tpr': [],
    'tpr_all': [],
}
for filename in os.listdir('result'):
    path = f'result/{filename}'
    _, w, t, c, end = filename.split('-')
    with open(path) as fp:
        for line in fp:
            if line.startswith('Time taken for tests'):
                pattern = re.compile("Time taken for tests:   (.*) seconds")
                total_time = float(pattern.findall(line)[0])
            if line.startswith('Requests per second'):
                pattern = re.compile("Requests per second:    (.*) \[#\/sec] \(mean\)")
                rps = float(pattern.findall(line)[0])
            if line.endswith('[ms] (mean)\n'):
                pattern = re.compile("Time per request:       (.*) \[ms\] \(mean\)")
                tpr = float(pattern.findall(line)[0]) / 1000
            if line.endswith('(mean, across all concurrent requests)\n'):
                pattern = re.compile("Time per request:       (.*) \[ms\] \(mean, across all concurrent requests\)")
                tpr_all = float(pattern.findall(line)[0]) / 1000
    data['name'].append(filename)
    data['num_worker'].append(w)
    data['num_threads'].append(t)
    data['worker_class'].append(c)
    data['endpoint'].append(end)
    data['total_time'].append(total_time)
    data['rps'].append(rps)
    data['tpr'].append(tpr)
    data['tpr_all'].append(tpr_all)

In [5]:
from pandas import DataFrame

In [76]:
df = DataFrame(data, columns=['name', 'worker_class', 'num_worker', 'num_threads', 'endpoint', 'total_time', 'tpr', 'tpr_all', 'rps'])
df = df.sort_values(['worker_class', 'num_worker', 'num_threads', 'endpoint'], ascending=[False, True, True, True])
df.head()

,name,worker_class,num_worker,num_threads,endpoint,total_time,tpr,tpr_all,rps
65,test-1-1-sync-compute,sync,1,1,compute,5.013,2.506625,0.156664,6.38
75,test-1-1-sync-request,sync,1,1,request,120.306,60.153054,3.759566,0.27
1,test-1-1-sync-sleep,sync,1,1,sleep,96.092,48.046143,3.002884,0.33
0,test-1-2-sync-compute,sync,1,2,compute,5.372,2.685846,0.167865,5.96
12,test-1-2-sync-request,sync,1,2,request,60.646,30.323084,1.895193,0.53


- 비교하려는 대상: class, num_worker, num_threads
- 엔드포인트별로, total_time, tpr, tpr_all, rps별로 12개
- 가로축은 [1, 2, 4] * [1, 2, 4] = 9개
- 각 포인트별로 버티컬 바 그래프 3개(class별로)

In [30]:
import plotly.plotly as py
from plotly.graph_objs import Bar, Layout, Figure

In [65]:
def matrix(row):
    return f'{row["num_worker"]}*{row["num_threads"]}'

def get_df(base, name):
    df = base[base.worker_class == name]
    return df.assign(matrix=df.apply(matrix, axis=1))

In [64]:
import plotly.offline as py

In [79]:
workers = ['sync', 'gevent', 'gaiohttp']
columns = [
    'total_time',
    'rps',
    'tpr',
    'tpr_all',
]
colors = ['#ffcdd2', '#a2d5f2', '#c6ffd2']
y_labels = [
    'Time taken for tests [sec]',
    'Requests per second [sec]',
    'Time per request [sec]',
    'Time per request, across all concurrent requests [sec]',
]

ends = ['sleep', 'request', 'compute']

In [80]:
end = ends[2]
base_df = df[df['endpoint'] == end]

def bar(base, worker_class, column, color):
    df = get_df(base, worker_class)
    return Bar(
        x=df.matrix,
        y=df[column],
        text=df[column].apply(int),
        textposition='auto',
        name=f'{worker_class}.{column}',
        marker=dict(color=color))

for col, label in zip(columns, y_labels):
    data = []
    for w, c in zip(workers, colors):
        data.append(bar(base_df, w, col, c))
    if col == 'rps':
        text = 'higher'
    else:
        text = 'lower'
    title = f'{end} * {col}'
    layout = Layout(title=f'{title} ({text} is better)',
                    xaxis=dict(title='worker process * threads'),
                    yaxis=dict(title=label))
    fig = Figure(data=data, layout=layout)


    py.plot(
        fig,
        show_link=False,
        filename=f'{title}.html',
        output_type='file',
        image='svg',
        image_filename=title,
        image_width=1600,
        image_height=900)